In [1]:
import calendar
import pandas as pd
import numpy as np
import io
import os
from pandas import read_csv
import numpy as np # linear algebra
import random as rd # generating random numbers
import datetime # manipulating date formats
import time
import ast
from math import sqrt
# Viz
import seaborn as sns # for prettier plots
import matplotlib.pyplot as plt
%matplotlib inline
# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.ar_model import AutoReg
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
import seaborn; seaborn.set()
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import warnings
warnings.filterwarnings('ignore')
import json
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import performance_metrics
import utils

In [2]:
df=pd.read_csv("/bigdata/tara/ind_london_2018_sample.csv",index_col=0)

In [3]:
df_station=pd.read_csv('SampleData/sample_stations.csv',index_col=0)
df_station.head()

,ucl_id,operator_intid,operator_altid,operator_name,lat,lon,initial_bikes,initial_size,curr_bikes,curr_size,created_dt,updated_dt,neighbors_1,neighbors_2,station_id,change_count
0,12,12,12,"Malet Street, Bloomsbury",51.521681,-0.130432,25,40,26,49,2010-08-06 01:00:00,2020-02-14 17:38:02,"[364, 287]","[364, 287, 88, 19, 796]",12,38553
1,116,116,116,"Little Argyll Street, West End",51.514500,-0.141424,12,21,5,21,2010-08-06 01:00:00,2020-02-14 17:38:02,"[349, 159]","[159, 349, 313, 106, 141]",116,37611
3,32,32,32,"Leonard Circus , Shoreditch",51.524696,-0.084439,17,21,9,43,2010-08-06 01:00:00,2020-02-14 17:38:02,[323],"[323, 73, 58, 3, 319]",32,40529
6,195,195,195,"Milroy Walk, South Bank",51.507244,-0.106238,3,30,19,30,2010-08-06 01:00:00,2020-02-14 17:38:02,"[839, 230, 240, 792]","[839, 240, 230, 792, 420]",195,36805
7,66,66,66,"Holborn Circus, Holborn",51.517950,-0.108657,39,39,26,40,2010-08-06 01:00:00,2020-02-14 17:38:02,"[546, 67, 835]","[546, 67, 835, 84, 112]",66,49363


In [4]:
df_exg=pd.read_csv('SampleData/exg_df.csv',index_col=0)

df_exg.head()

,timestamp,trip_count,temp,pressure,wind_speed,rain_1h,humidity,clouds_all,Clear,Clouds,...,Smoke,Snow,Thunderstorm,is_weekend,is_holiday,trip_log,is_non_workday,hour,is_rushhour,is_day
0,2018-01-01 00:00:00,754,280.27,996,9,0.0,70,68,0,1,...,0,0,0,0,1,6.626718,1,0,0,0
1,2018-01-02 00:00:00,90,279.16,1009,4,0.0,75,48,0,1,...,0,0,0,0,0,4.510860,0,0,0,0
2,2018-01-03 00:00:00,70,285.27,988,11,3.0,76,92,0,0,...,0,0,0,0,0,4.262680,0,0,0,0
3,2018-01-04 00:00:00,121,280.77,1001,6,0.0,75,40,0,1,...,0,0,0,0,0,4.804021,0,0,0,0
4,2018-01-05 00:00:00,94,278.85,993,3,0.3,81,92,0,0,...,0,0,0,0,0,4.553877,0,0,0,0


In [5]:
start_dates=['2018-01-01','2018-02-01','2018-03-01','2018-04-01','2018-05-01','2018-06-01','2018-07-01','2018-08-01',
             '2018-09-01','2018-10-01','2018-11-01','2018-12-01']
end_dates=['2018-01-31','2018-02-28','2018-03-31','2018-04-30','2018-05-31','2018-06-30','2018-07-31','2018-08-31'
           ,'2018-09-30','2018-10-31','2018-11-30','2018-12-31']


In [6]:
freq=60 #the original freq of data is 2 minutes interval
time_offset=60 #for how many minutes ahead we want to predict
#forecast_steps=int(time_offset/freq) #number of predictions that we need to predict time_offset ahead
test_days=7 #the number of days for test
test_size=int(test_days*24*60/freq) 
forecast_steps=int(time_offset/freq)
window_size=int(4*60/freq)
Stations=list(df_station['ucl_id'])

In [7]:
xtest_size=164

In [8]:
def remove_seasonality(temp_df,test_size):
    df_temp=temp_df.copy()
    avgs=df_temp[0:-test_size].groupby([df_temp[0:-test_size].index.dayofweek,df_temp[0:-test_size].index.time]).mean()
    for name, column in (df_temp.iteritems()):
        df_temp[name+'_avg'] = df_temp.index.map(lambda d: avgs.loc[(d.dayofweek,d.time()),name])
        df_temp[name] = df_temp[name] - df_temp[name+'_avg']
    
        #df_temp=df_temp.drop(name+'_avg',axis=1)
    n_cols=len(list(df_temp.columns))
    df1 = df_temp.iloc[:, :int(n_cols/2)] #var_df
    df2 = df_temp.iloc[:, int(n_cols/2):] #vard_df_avg
    return df1,df2

In [9]:
def add_seasonality(df1,df2):
    df_1=df1.copy()
    df_2=df2.copy()
    for name, column in (df_1.iteritems()):
        
        df_1[name] = df_1[name] + df_2[name.split('_forecast_')[0]+'_avg']
    return df_1
        

In [10]:
def Sarima(p,d,q,station_id):
    global i
    global summaries
    global model_times
    forecasts=[]
    tests=[]
    df_exg_history= data_exg_train.copy()
    df_history=df_train.copy()
    t0=test_size ##neded for reverse the diff
    
    t0=test_size ##neded for reverse the diff  
    
    # walk-forward validation
    
    
    for t in range(0,len(df_test),window_size):
        result={}
        predictions = list()
        #because of the warning for frequency
        df_history.index = pd.DatetimeIndex(df_history.index.values,
                                   freq=df_history.index.inferred_freq)
        # fit model
        start=time.time()
        model = SARIMAX(df_history[-len(df_train):],order=(p,d,q),seasonal_order=(0,0,0,0),
                        enforce_stationarity=False, enforce_invertibility=False,
                        exog=df_exg_history[-len(df_train):], time_varying_regression=True, mle_regression=False)

        model_fit = model.fit()
        end=time.time()
        model_times.append(end-start)
#         print(model_fit.summary())
        results_summary=model_fit.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_summary=pd.read_html(results_as_html, header=0, index_col=0)[0]
        summaries.append(df_summary)
        #print(df_summary)
        yhat = model_fit.forecast(steps=window_size,exog =data_exg_test[t:t+window_size])
     
        ### update indogenous and exogenous dataframes
        df_history = df_history.append(df_test[t:t+window_size])
        df_exg_history=df_exg_history.append(data_exg_test[t:t+window_size])
        
        #df_forecast = pd.DataFrame(yhat, index=df_test.index[t:t+window_size], columns=df_train.columns + '_forecast_'+str(d))
        predictions.extend(yhat)

        if t==xtest_size:
            df_forecast = pd.DataFrame(predictions, index=data.index[-test_size+t:], columns=data.columns + '_forecast_'+str(d))
        else:
            df_forecast = pd.DataFrame(predictions, index=data.index[-test_size+t:-test_size+t+window_size], 
                                       columns=data.columns + '_forecast_'+str(d))
        
        #######bring forecast data back to normal scale 
       
        df_forecast= add_seasonality(df_forecast,df_test_avg[t:t+window_size])  
        #df_forecast=utils.invert_transformation_forecast(data_original, df_forecast,t0,diff_count=d)
        df_forecast=utils.inverse_normalize(df_forecast,total_dock)
        
        ####bring test data back to normal scale 
        df_actual= add_seasonality(df_test[t:t+window_size],df_test_avg[t:t+window_size])
       
        #df_actual= utils.invert_transformation_test(data_original, df_test,t0,diff_count=d)
        df_actual=utils.inverse_normalize(df_actual,total_dock) 
        
       
 
        t0=test_size +window_size
        
        df_forecast= utils.forecast_truncate(df_forecast,total_dock)
        ######################write to file
        forecasts.extend(list(df_forecast.iloc[:,0].values))
        tests.extend(list(df_actual.iloc[:,0].values))

        if ((t+window_size)%(6*window_size) == 0):
                result['day']=df_forecast.index[1].dayofweek
                result['month']=df_forecast.index[1].month
                result['station_id']=station_id
    #             result['predict']=list(df_forecast.iloc[:,0].values)
    #             result['test']=list(df_actual.iloc[:,0].values)
                result['predict']=forecasts
                result['test']=tests

                with open('report/test/regressors/SARIMAX0/'+'sarima_result_'+str(station_id)+'_'+str(i)+'.json', 'w+') as f:
                #with open('report/SARIMA/'+str(freq)+'minutes/sarima_result_'+str(station_id)+'_'+str(i)+'.json', 'w+') as f:
                    f.write(json.dumps(result))
                i=i+1
                forecasts=[]
                tests=[]
#         break
    #return df_forecast

In [11]:
start=time.time()

summaries=[]
model_times=list()
i=0
for t in range(0,1):
  
    start_date=start_dates[t]
    end_date=end_dates[t]        
    print(start_date)
    data_exg_original=utils.clean_exg_df(df_exg,start_date,end_date,freq)
    for station_id in Stations:
        
        
        
        total_dock=df[df['operator_id']==int(station_id)]['total_docks'].values[0]
        data=df[df['operator_id']==station_id]
        #data prepration       
        
        data=utils.clean_df(data,station_id,start_date,end_date,freq)
        
        data_exg=data_exg_original[data_exg_original.index.isin(data.index)]
        data=utils.normalize(data,total_dock)
        anomalies=utils.anomaly_detection(data,freq)
        data=utils.anomaly_removal(anomalies,data)
        
        data,data_avg=remove_seasonality(data,test_size)
        #checking for stationarity
        a,d=utils.make_Stationary(data)
        
        
        
        df_train, df_test = data[0:len(data)-test_size], data[len(data)-test_size:]
        df_train_avg, df_test_avg = data_avg[0:len(data_avg)-test_size], data_avg[len(data_avg)-test_size:]
        data_exg_train,data_exg_test= data_exg[0:len(data_exg)-test_size], data_exg[len(data_exg)-test_size:]
        if d>0:
            print(d)
        #apply the model and get the result
        Sarima(2,d,0,station_id)
#         break
#     break


2018-01-01
1


In [38]:
#time
with open('sarimax_model_time.txt', 'w+') as f:
        for item in model_times:
            f.write("%s\n" % item)

In [40]:
df_summaries=pd.concat(summaries)
df_summaries.to_csv('sarimax_summary.csv')